In [3]:
import sys
import os
import pandas as pd
import shutil

In [4]:
# Declarring paths
nodes_dir = "C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/"
dec_nodes_dir = "C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/"

In [5]:
labels = pd.read_csv("C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/december_data_labels.csv")

In [18]:
df = labels 

In [19]:
df['original_image_path'] = dec_nodes_dir + 'P ' + df['patient ID'].astype(str) + '/' + df['Original Image Name']
df['original_image_path'].iloc[0]

'C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/P 101/Clipboard_03-15-2025_17'

In [21]:
# Verify paths exist and rename files from Original Image Name to Image ID (find file regardless of extension)
# Recompute paths to be sure they match destination after copying
import glob

df['original_image_path'] = dec_nodes_dir + 'P ' + df['patient ID'].astype(str) + '/' + df['Original Image Name']

# Helper to find actual file (with extension) given a path without extension
def find_actual_file(path_without_ext):
    # If a file exists exactly (maybe Original Image Name already had extension)
    if os.path.exists(path_without_ext):
        return path_without_ext
    # Try matches with any extension
    matches = glob.glob(path_without_ext + '.*')
    if len(matches) == 1:
        return matches[0]
    elif len(matches) > 1:
        # Prefer common image extensions if multiple matches
        preferred = ['.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp', '.dcm']
        for ext in preferred:
            for m in matches:
                if m.lower().endswith(ext):
                    return m
        # If no preferred extension found, return first and warn
        print(f"Warning: multiple matches for {path_without_ext}, using first: {matches[0]}")
        return matches[0]
    else:
        return None

# Find actual files and report
df['found_path'] = df['original_image_path'].apply(find_actual_file)
df['path_exists'] = df['found_path'].notnull()
print(f"Paths found: {df['path_exists'].sum()} / {len(df)}")
missing = df.loc[~df['path_exists'], 'original_image_path']
if not missing.empty:
    print("Missing files (showing up to 20):")
    print(missing.head(20).to_list())

# Rename files in destination to use Image ID (preserve found extension)
renamed = 0
skipped = 0
for idx, row in df.iterrows():
    found = row['found_path']
    if not isinstance(found, str) or not os.path.exists(found):
        skipped += 1
        continue
    orig_fname = os.path.basename(found)
    ext = os.path.splitext(orig_fname)[1]
    new_fname = str(row['Image ID']) + ext
    new_path = os.path.join(os.path.dirname(found), new_fname)
    if os.path.exists(new_path):
        print(f"Target already exists, skipping: {new_path}")
        skipped += 1
        continue
    os.rename(found, new_path)
    renamed += 1
    # Update DataFrame so names/paths reflect the rename
    df.at[idx, 'Original Image Name'] = new_fname
    df.at[idx, 'original_image_path'] = os.path.join(os.path.dirname(found), new_fname)
    df.at[idx, 'found_path'] = new_path

print(f"Renamed: {renamed}, Skipped (missing or target exists): {skipped}")

Paths found: 304 / 600
Missing files (showing up to 20):
['C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/P 101/Clipboard_03-15-2025_17', 'C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/P 101/Clipboard_03-15-2025_18', 'C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/P 101/Clipboard_03-15-2025_19', 'C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/P 101/Clipboard_03-15-2025_20', 'C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/P 101/Clipboard_03-15-2025_21', 'C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/P 102/Clipboard_03-15-2025_36', 'C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/P 102/Clipboard_03-15-

In [22]:
nodes_df = df[df['Image type'] == 'Node']

In [23]:
nodes_df

,S.No.,patient ID,Age,No. of nodes,Histopath,ER,PR,Her 2 neu,Ki67,molecularsubtype,histopathology,FNAC/BIOPSY,Additional comments,Original Image Name,Image ID,Image type,Comments,original_image_path,found_path,path_exists
0,1,101,27.0,1.0,IBC NST,8,6,Negative 0,25,Luminal B,METASTATIC,fnac,NaN,Clipboard_03-15-2025_17,P_101_1,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,None,False
1,1,101,27.0,1.0,IBC NST,8,6,Negative 1,25,Luminal B,METASTATIC,fnac,NaN,Clipboard_03-15-2025_18,P_101_2,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,None,False
2,1,101,27.0,1.0,IBC NST,8,6,Negative 2,25,Luminal B,METASTATIC,fnac,NaN,Clipboard_03-15-2025_19,P_101_3,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,None,False
3,1,101,27.0,1.0,IBC NST,8,6,Negative 3,25,Luminal B,METASTATIC,fnac,NaN,Clipboard_03-15-2025_20,P_101_4,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,None,False
4,1,101,27.0,1.0,IBC NST,8,6,Negative 4,25,Luminal B,METASTATIC,fnac,NaN,Clipboard_03-15-2025_21,P_101_5,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,92,195,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REACTIVE HYPERPLASIA/BENIGN,FNAC,NaN,IMG-0022-00008,P_195_8,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,None,False
593,92,195,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REACTIVE HYPERPLASIA/BENIGN,FNAC,NaN,IMG-0022-00009,P_195_9,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,None,False
594,92,195,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REACTIVE HYPERPLASIA/BENIGN,FNAC,NaN,IMG-0022-00010,P_195_10,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,None,False
595,92,195,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REACTIVE HYPERPLASIA/BENIGN,FNAC,NaN,IMG-0022-00011,P_195_11,Node,NaN,C:/HII Lab/Breast Cancer/Breast_Cancer_Ultraso...,None,False


In [24]:
# Move files matching Image ID (any extension) to december_nodes_images inside dec_nodes_dir
import glob
import shutil

target_folder = os.path.join(nodes_dir, 'december_nodes_images')
os.makedirs(target_folder, exist_ok=True)

moved = 0
skipped = 0
missing = []

# Helper to find actual file (with extension) given a base path without extension
def find_actual_file_noext(base_path):
    # If exact path exists
    if os.path.exists(base_path):
        return base_path
    matches = glob.glob(base_path + '.*')
    if len(matches) == 1:
        return matches[0]
    elif len(matches) > 1:
        preferred = ['.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp', '.dcm']
        for ext in preferred:
            for m in matches:
                if m.lower().endswith(ext):
                    return m
        print(f"Warning: multiple matches for {base_path}, using first: {matches[0]}")
        return matches[0]
    else:
        return None

# Iterate rows (use nodes subset if you prefer)
for idx, row in nodes_df.iterrows():
    base = os.path.join(dec_nodes_dir, 'P ' + str(row['patient ID']), str(row['Image ID']))
    found = find_actual_file_noext(base)
    if found is None:
        missing.append(base)
        continue
    # If already in target, skip
    if os.path.dirname(found) == target_folder:
        skipped += 1
        continue
    dest_path = os.path.join(target_folder, os.path.basename(found))
    if os.path.exists(dest_path):
        print(f"Destination already exists {dest_path}, skipping")
        skipped += 1
        continue
    shutil.copy(found, dest_path)
    moved += 1
    nodes_df.at[idx, 'moved_path'] = dest_path

print(f"Moved: {moved}, Skipped: {skipped}, Missing: {len(missing)}")
if missing:
    print("Missing examples (up to 20):", missing[:20])

C:\Users\rautc\AppData\Local\Temp\ipykernel_4072\684591320.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_df.at[idx, 'moved_path'] = dest_path


Moved: 293, Skipped: 0, Missing: 1
Missing examples (up to 20): ['C:/HII Lab/Breast Cancer/Breast_Cancer_Ultrasound_Project_AIIMS/data/nodes_data/IITM/Anonymised Axilla/P 177\\P_177_2']


In [26]:
nodes_df.to_csv('december_nodes_labels.csv')

In [1]:
#Create Masks for nodes images in cropped_images folder

In [2]:
import sys 
sys.path.append('../utils')

In [3]:
from image_processing_helper import *
from radiomics_helper import *

In [7]:
import os
import cv2  # if you're using OpenCV, adjust if different

cropped_images_folder = "../data/nodes_data/december_nodes_images/Cropped_Images_Folder"
cropped_masks_folder  = "../data/nodes_data/december_nodes_images/cropped_images_masks_folder"

os.makedirs(cropped_masks_folder, exist_ok=True)

for fname in os.listdir(cropped_images_folder):
    if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".tif", ".tiff")):
        continue

    name, ext = os.path.splitext(fname)
    out_name = f"{name}_mask{ext}"
    out_path = os.path.join(cropped_masks_folder, out_name)

    # Skip if mask already exists
    if os.path.exists(out_path):   # or os.path.isfile(out_path)
        continue

    img_path = os.path.join(cropped_images_folder, fname)
    img = read_as_grayscale(img_path)

    img_mask = manual_mask_segmentation(img)

    cv2.imwrite(out_path, img_mask)

Draw the contour using left mouse button. Press 's' to finish and return the mask.
